In [69]:
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [29]:
file_path = os.path.join('..', 'dataset', 'train.csv')
file_path_response_format = os.path.join('..', 'dataset', 'gender_submission.csv')  
file_path_test = os.path.join('..', 'dataset', 'test.csv')

train_df = pd.read_csv(file_path)
df_response_format = pd.read_csv(file_lath_response_format)
df_test = pd.read_csv(file_path_test)

In [30]:
print("\n Train Dataset information:")
print(train_df.info())


 Train Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [15]:
print("\n Test Dataset information:")
print(df_test.info())


 Test Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB
None


In [16]:
print("\n Response Formt Dataset information:")
print(df_response_format.info())


 Response Formt Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  418 non-null    int64
 1   Survived     418 non-null    int64
dtypes: int64(2)
memory usage: 6.7 KB
None


In [8]:
print("\nDataset descriptive statistics:")
print(df.describe())


Dataset descriptive statistics:
       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  


In [9]:
print("\nNumber of missing values per column:")
print(df.isnull().sum())


Number of missing values per column:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [59]:
y = train_df['Survived']

numerical_features = train_df.select_dtypes(include=['int64', 'float64']).drop(['Survived', 'PassengerId'], axis=1).columns

X_numerical = train_df[numerical_features].copy() 

print("Number of NaN values per numerical feature before dropping:")
print(X_numerical.isnull().sum())

nan_rows_index = X_numerical[X_numerical.isnull().any(axis=1)].index

X_numerical_cleaned = X_numerical.dropna()

y_cleaned = y.drop(nan_rows_index)

print("\nShape of X_numerical before dropping NaN:", X_numerical.shape)
print("Shape of y before dropping NaN:", y.shape)
print("Shape of X_numerical after dropping NaN:", X_numerical_cleaned.shape)
print("Shape of y after dropping NaN:", y_cleaned.shape)

Number of NaN values per numerical feature before dropping:
Pclass      0
Age       177
SibSp       0
Parch       0
Fare        0
dtype: int64

Shape of X_numerical before dropping NaN: (891, 5)
Shape of y before dropping NaN: (891,)
Shape of X_numerical after dropping NaN: (714, 5)
Shape of y after dropping NaN: (714,)


In [60]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X_numerical_cleaned)

X_scaled_df = pd.DataFrame(X_scaled, columns=numerical_features)

print("\nScaled numerical features (StandardScaler) after dropping NaN:")
print(X_scaled_df.head())

print("\nShape of scaled features (X_scaled):", X_scaled.shape)
print("Shape of the cleaned label (y_cleaned):", y_cleaned.shape)


Scaled numerical features (StandardScaler) after dropping NaN:
     Pclass       Age     SibSp     Parch      Fare
0  0.911232 -0.530377  0.524570 -0.505895 -0.518978
1 -1.476364  0.571831  0.524570 -0.505895  0.691897
2  0.911232 -0.254825 -0.551703 -0.505895 -0.506214
3 -1.476364  0.365167  0.524570 -0.505895  0.348049
4  0.911232  0.365167 -0.551703 -0.505895 -0.503850

Shape of scaled features (X_scaled): (714, 5)
Shape of the cleaned label (y_cleaned): (714,)


In [64]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y_cleaned, test_size=0.2, random_state=42)

# Initialize and train the XGBoost classifier
xgb_classifier = XGBClassifier(random_state=42)
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of XGBoost on the test set: {accuracy:.4f}")


Accuracy of XGBoost on the test set: 0.6923


In [72]:
# جدا کردن ویژگی (X) و برچسب (y)
X = train_df.drop('Survived', axis=1)
y = train_df['Survived']

# 1. مدیریت مقادیر از دست رفته
X['Age'] = X['Age'].fillna(X['Age'].mean())
X['Embarked'] = X['Embarked'].fillna(X['Embarked'].mode()[0])
# استخراج حرف اول Cabin
X['Cabin_initial'] = X['Cabin'].str[0].fillna('Unknown')

X['Cabin_known'] = X['Cabin'].apply(lambda x: 0 if pd.isna(x) else 1)
X.drop('Cabin', axis=1, inplace=True)  # حذف ستون اصلی Cabin بعد از ایجاد ویژگی Cabin_initial و Cabin_known

# 2. مهندسی ویژگی
X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
X['IsAlone'] = X['FamilySize'].apply(lambda x: 1 if x == 1 else 0)

def extract_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

X['Title'] = X['Name'].apply(extract_title)
X['Title'].replace(['Lady', 'Countess','Capt', 'Col',
                    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare', inplace=True)
X['Title'].replace('Mlle', 'Miss', inplace=True)
X['Title'].replace('Ms', 'Miss', inplace=True)
X['Title'].replace('Mme', 'Mrs', inplace=True)
X.drop('Name', axis=1, inplace=True)  # حذف ستون اصلی Name بعد از استخراج Title
X.drop('Ticket', axis=1, inplace=True)  # حذف ستون Ticket

# 3. انتخاب ویژگی‌ها برای پیش‌پردازش
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare', 'FamilySize']
categorical_features = ['Sex', 'Pclass', 'Embarked', 'Cabin_known', 'IsAlone', 'Title', 'Cabin_initial']

# 4. ایجاد Pipeline برای پیش‌پردازش
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# 5. تقسیم داده‌ها به آموزش و آزمایش
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. ایجاد و آموزش مدل XGBoost در یک Pipeline
xgb_model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', XGBClassifier(random_state=42))])

xgb_model.fit(X_train, y_train)

# 7. پیش‌بینی و ارزیابی مدل
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of XGBoost with feature engineering: {accuracy:.4f}")


Accuracy of XGBoost with feature engineering: 0.7933


/var/folders/55/ckz71qcd6zz56zdm9qthg9v40000gn/T/ipykernel_76390/3551535491.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['Title'].replace(['Lady', 'Countess','Capt', 'Col',
